ابتدا کتابخانه های لازم را ایمپورت میکنیم.

In [1]:
import numpy as np
import shutil
from zipfile import ZipFile
import matplotlib.pyplot as plt
import os
from PIL import Image
import cv2
import glob
import random

سپس گوگل درایو را mount میکنیم.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


حال پوشه مورد نظر برای ترین کردن مدل را میسازیم.

In [3]:
data_root = '/content/data'

if not os.path.isdir(data_root):
    os.mkdir(data_root)

در اینجا لیبل هایی که در داخل گوگل درایو گذاشتیم را میخوانیم و همچنین فیلم مربوطه را خوانده و فریم های آن را جدا کرده و پس از آن آنها را در جای مناسب مطابق به مدل یولو قرار میدهیم.

In [4]:
vidcap = cv2.VideoCapture('/content/drive/MyDrive/3Birds_inCage_stable(easy) (2).mp4')
success,image = vidcap.read()

count = 0

feameLabel = '/content/data/obj'
label_cvat_root = '/content/drive/MyDrive/obj_train_data'
if not os.path.isdir(feameLabel):
    os.mkdir(feameLabel)
while success:
  if( count >= 1400):
    frame_name = f"frame_00{count}"

    frame_path = os.path.join(feameLabel, f"{frame_name}.jpg")
    cv2.imwrite(frame_path, image)     # JPEG
    label_path = os.path.join(feameLabel, f"{frame_name}.txt")
    label_cvat = os.path.join(label_cvat_root, f"{frame_name}.txt")

    with open(label_cvat) as f:
      with open(label_path, "w") as f1:
        for line in f:
            f1.write(line)
  success,image = vidcap.read()
  count += 1

In [5]:
# preparing the folder structure

full_data_path = '/content/data/obj/'
extension_allowed = '.jpg'
split_percentage = 88

images_path = '/content/data/images/'
if os.path.exists(images_path):
    shutil.rmtree(images_path)
os.mkdir(images_path)

labels_path = '/content/data/labels/'
if os.path.exists(labels_path):
    shutil.rmtree(labels_path)
os.mkdir(labels_path)

training_images_path = images_path + 'training/'
validation_images_path = images_path + 'validation/'
training_labels_path = labels_path + 'training/'
validation_labels_path = labels_path +'validation/'

os.mkdir(training_images_path)
os.mkdir(validation_images_path)
os.mkdir(training_labels_path)
os.mkdir(validation_labels_path)

files = []

ext_len = len(extension_allowed)

for r, d, f in os.walk(full_data_path):
    for file in f:
        if file.endswith(extension_allowed):
            strip = file[0:len(file) - ext_len]
            files.append(strip)

random.shuffle(files)

size = len(files)

split = int(split_percentage * size / 100)

# copying training data
for i in range(split):
    strip = files[i]

    image_file = strip + extension_allowed
    src_image = full_data_path + image_file
    shutil.copy(src_image, training_images_path)

    annotation_file = strip + '.txt'
    src_label = full_data_path + annotation_file
    shutil.copy(src_label, training_labels_path)

# copying validation data
for i in range(split, size):
    strip = files[i]

    image_file = strip + extension_allowed
    src_image = full_data_path + image_file
    shutil.copy(src_image, validation_images_path)

    annotation_file = strip + '.txt'
    src_label = full_data_path + annotation_file
    shutil.copy(src_label, validation_labels_path)

In [6]:
# cloning yolov5
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt  # install dependencies

Cloning into 'yolov5'...
remote: Enumerating objects: 15814, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 15814 (delta 9), reused 23 (delta 2), pack-reused 15768
Receiving objects: 100% (15814/15814), 14.64 MiB | 18.81 MiB/s, done.
Resolving deltas: 100% (10821/10821), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.9/626.9 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.3 MB/s eta 0:00:00


In [ ]:
!pwd

/content/yolov5


In [ ]:
# number_of_epochs = 50  # desired in question

# but due to the limit of time we use following:
# number_of_epochs = 2  # limited time :(

In [7]:
!python train.py --img 640 --batch 2 --epochs 2 --data /content/dataset.yaml --weights yolov5s.pt

train: weights=yolov5s.pt, cfg=, data=/content/dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=2, batch_size=2, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-193-g485da42 Python-3.10.12 torch-2.0.1+cu118 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, tra